# Getting started
This notebook illustrates the basic usage of Vis4D. We will visualize a COCO image with detections from Faster RCNN.

First, import the necessary components from the library


In [1]:

from vis4d.model.detect.faster_rcnn import FasterRCNN

from vis4d.run.data.detect import default_test_pipeline
from vis4d.data.datasets.coco import COCO
from vis4d.data.const import CommonKeys
from vis4d.vis.functional import imshow_bboxes
import os

os.chdir("/home/tobfischer/PycharmProjects/vist/dev")


/itet-stor/tobfischer/net_scratch/conda_envs/vis4d-dev/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Now, let's create the dataset and fetch the image from it.

In [2]:

dataset = COCO("tests/test_data/coco_test/", split="train")
test_dataloader = default_test_pipeline(dataset, 1, 1, (800, 1333))[0]
batch = next(iter(test_dataloader))
inputs, images_hw = (
    batch[CommonKeys.images],
    batch[CommonKeys.input_hw],
)

Loading COCODataset(root=tests/test_data/coco_test/, split=train, use_pascal_voc_cats=False) takes 0.00 seconds.


Next, we can initialize and run the model on it.

In [3]:

faster_rcnn = FasterRCNN(num_classes=80, weights="mmdet")

faster_rcnn.eval()
dets = faster_rcnn(inputs, images_hw, original_hw=images_hw)

/itet-stor/tobfischer/net_scratch/conda_envs/vis4d-dev/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/itet-stor/tobfischer/net_scratch/conda_envs/vis4d-dev/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth


Finally, let's visualize the result.

In [4]:
imshow_bboxes(inputs[0], dets["boxes2d"][0], dets["boxes2d_scores"][0], dets["boxes2d_classes"][0])

TypeError: min() received an invalid combination of arguments - got (out=NoneType, axis=tuple, ), but expected one of:
 * ()
 * (Tensor other)
 * (int dim, bool keepdim)
      didn't match because some of the keywords were incorrect: out, axis
 * (name dim, bool keepdim)
      didn't match because some of the keywords were incorrect: out, axis
